In [1]:
    if __name__ == '__main__': 
        %reload_ext pidgin
        from pidgin.docs.references import *

In [2]:
`import pidgin, pidgin.specification.markdown` defines custom renderers for `import\` `mistune, doctest, pygments, graphviz`.

`import mistune`, `pidgin` uses `mistune` to __tangle__ and __weave__ Markdown source code.

In [3]:
    import ast, jinja2, sys, nbconvert, nbformat, contextlib, collections, IPython, mistune, doctest, re, traitlets, textwrap, types, typing as t, importlib, importnb,  jinja2, builtins, jsonschema
    from pidgin import tangle, shell, formatter

In [4]:
    class Markdown(mistune.Renderer, mistune.Markdown, shell.Shell):
        run = traitlets.Bool(default_value=True)
        def __init__(Markdown, *args, **kwargs):
            shell.Shell.__init__(Markdown, *args, **kwargs)
            mistune.Renderer.__init__(Markdown)
            mistune.Markdown.__init__(Markdown, renderer=Markdown, block=BlockLexer, inline=InlineLexer)
            Markdown.block.shell = Markdown
            Markdown._codespan_buffer = []
            Markdown._block_code_buffer = []

        def __enter__(Markdown): 
            Markdown.run = False
            return Markdown
        
        def __exit__(Markdown, *e): Markdown.run = True
            
        def codespan(Markdown, text, lang=None):
            Markdown._codespan_buffer.append(text)
            if not text.endswith('\\'):
                Markdown.run_literal('\n'.join(Markdown._codespan_buffer))
                Markdown._codespan_buffer = []
            text = text.rstrip('\\')
            return F"""<span{Markdown.pygment(text, prestyles="display: inline-block; vertical-align: middle;", nobackground=True).strip()[4:-4]}span>"""
        
        def pygment(Markdown, text, lang=None, **kwargs):
            lang = lang or 'python'
            import pygments
            return pygments.highlight(text, pygments.lexers.get_lexer_by_name(lang, stripall=True), pygments.formatters.html.HtmlFormatter(noclasses=True, **kwargs))
        
        def block_code(Markdown, text, lang=None, lineno=None):
            if lang is None: 
                Markdown._block_code_buffer.append(text)
                if not text.endswith('\\'):
                    Markdown.run_literal('\n'.join(Markdown._block_code_buffer))
                    Markdown._block_code_buffer = []
            return Markdown.pygment(text.rstrip('\\'), lang)
        
        def run_literal(Markdown, str):
            return Markdown.run and super().run_cell(textwrap.indent(str, ' '*4), silent=True)
        
        def output_doctest(self): return self.renderer.doctest(self.token['text'])
        
        def output_graphviz(self): return self.renderer.graphviz(self.token['text'])
        

        def _output_code(self):
            return self.renderer.block_code(self.token['text'], self.token['lang'], self.token['lineno'])
                
        def doctest(self, text): return super().block_code(text)
        def graphviz(self, text): 
            return __import__('graphviz').Source(text)._repr_svg_()

        
            #         def header(self, text, level, raw=None):
            #             return F"""<h{level} id="{__import__('slugify').slugify(text, lowercase=False)}">{text}</h{level}"""

In [5]:
    class DocTestGrammar:
`DocTestGrammar` adds `doctest` to `pidgin`'s `mistune.BlockGrammar`.
        
        doctest = doctest.DocTestParser._EXAMPLE_RE
        
    class DoctestLexer:
`DoctestLexer` adds a `"doctest"` token types type to the parsed __Markdown__.        

        def parse_doctest(self, m): self.tokens.append({'type': 'doctest', 'text': m.string[slice(*m.span())]})
            

In [8]:
### Learn more about [`doctest`][]

In [9]:
    class GraphvizGrammar:
`GraphvizGrammar` recognizes [graphviz syntax] in the `mistune.BlockGrammar`.        

        graphviz = re.compile("^(di){0,1}graph\s*\{(\{(\{(\{(\{(\{.*?\}|.)*?\}|.)*?\}|.)*?\}|.)*?\}|.)*?\}", re.DOTALL)
    class GraphvizLexer:
        def parse_graphviz(self, m): 
            self.tokens.append({'type': 'graphviz', 'text': m.string[slice(*m.span())]})   

In [10]:
> #### [what is graphviz?][]
>> Learn more about [graphviz][what is graphviz?] and its [syntax][graphviz syntax].

In [13]:
## Block and inline `mistune` lexers.

In [12]:
    class BlockLexer(GraphvizLexer, DoctestLexer, mistune.BlockLexer): 
        class grammar_class(GraphvizGrammar, DocTestGrammar, mistune.BlockGrammar): ...
        
        def __init__(BlockLexer, rules=None, **kwargs): super().__init__(rules, **kwargs)
        
        def parse(self, text, rules=None, *offsets)-> t.List[t.Dict]:
            from pidgin import transform_cell
            
            original = text.rstrip('\n')
            text = transform_cell.strip_blank_lines(original)
            if not rules: rules = self.default_rules

            def manipulate(text):
                for key in rules:
                    rule = getattr(self.rules, key)
                    m = rule.match(text)
                    if not m:
                        continue
                    getattr(self, 'parse_%s' % key)(m)
                    return m
                return False  # pragma: no cover

            while text:
                m = manipulate(text)                
                if m is not False:
                    offsets += len(m.group(0)) if offsets else 0,
                    text = text[offsets[-1] or len(m.group(0)):]
                    self.tokens[-1]['lineno'] = len(original[sum(offsets):].splitlines())
                    continue
                if text:  raise RuntimeError('Infinite loop at: %s' % text)
            return self.tokens
        
        def parse_block_code(self, m=None): 
            self.tokens[-1]['lang'] = super().parse_block_code(m) or ''
            
        def parse_block_quote(self, m):
            object = m.string[slice(*m.span())]
            if object not in self.shell.user_ns: self.shell.user_ns[object] = object
            super().parse_block_quote(m)
            
    for attr in "doctest graphviz".split():
        for key in "default_rules footnote_rules list_rules".split():[
            setattr(BlockLexer, key, list(BlockLexer.default_rules)), 
            getattr(BlockLexer, key).insert(getattr(BlockLexer, key).index('block_quote'), attr)]

In [11]:
    class InlineLexer(mistune.InlineLexer):
        def output_reflink(self, m):
            key = mistune._keyify(m.group(2) or m.group(1)).lower()
            if key in self.renderer.user_ns:
                m2 = mistune.BlockGrammar.def_links.match(F"[xxx]: {self.renderer.user_ns[key]}")
                ret = self.links[key] = dict(link=m2.group(2), title=m2.group(3))
                return self._process_link(m, ret['link'], ret['title'])
            return super().output_reflink(m)

        def output_emphasis(self, m):
            object = m.string[slice(*m.span())]
            if object not in self.renderer.user_ns:
                self.renderer.user_ns[object] = object
            text = m.group(2) or m.group(1)
            text = self.output(text)
            return self.renderer.emphasis(text)